# Analysis of the tone of reviews

Задача - классифицировать отзывы с IMDB на положительный / отрицательный только по тексту.

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment1.png">

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

In [1]:
!pip install torchtext
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

In [3]:
from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root="./data")

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:02<00:00, 40.1MB/s]


In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:
print(vars(train_data.examples[0]))

{'text': ['I', 'found', 'the', 'film', 'quite', 'expressive', ',', 'the', 'way', 'the', 'main', 'character', 'was', 'lost', 'but', 'at', 'the', 'same', 'much', 'more', 'clear', 'about', 'certain', 'things', 'in', 'life', 'than', 'people', 'who', 'mocked', 'him', '(', 'his', 'flatmate', 'for', 'example', ')', '.<br', '/><br', '/>he', 'was', 'tortured', 'and', 'you', 'loved', 'to', 'watch', 'him', 'being', 'tortured', '!', 'it', 'had', 'this', 'perverted', 'side', 'which', 'was', 'frightening', 'but', 'we', 'were', 'all', 'happy', 'to', 'see', 'him', 'come', 'out', 'of', 'the', 'misery', 'again', '.<br', '/><br', '/>it', 'was', 'like', 'a', 'game', 'character', 'or', 'pan', '-', 'man', 'through', 'a', 'mine', '-', 'land', 'or', 'to', 'enemy', 'and', 'we', 'love', 'to', 'watch', 'him', 'under', 'sniper', 'attack', 'or', 'fire', 'but', 'then', 'at', 'the', 'end', 'we', 'are', 'happy', 'to', 'see', 'him', 'survive', '...', '<', 'br', '/><br', '/', '>', '.'], 'label': 'pos'}


In [6]:
import random

test_data, valid_data = test_data.split(random_state=random.seed(SEED), split_ratio=0.5)

In [7]:
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
vars(LABEL.vocab)

{'freqs': Counter({'neg': 12500, 'pos': 12500}),
 'itos': ['neg', 'pos'],
 'stoi': defaultdict(None, {'neg': 0, 'pos': 1}),
 'unk_index': None,
 'vectors': None}

In [10]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 289838), (',', 275296), ('.', 236843), ('and', 156483), ('a', 156282), ('of', 144055), ('to', 133886), ('is', 109095), ('in', 87676), ('I', 77546), ('it', 76545), ('that', 70355), ('"', 63329), ("'s", 61928), ('this', 60483), ('-', 52863), ('/><br', 50935), ('was', 50013), ('as', 43508), ('with', 42807)]


* stoi (string to int)
* itos (int to string)

In [11]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [12]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [13]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device,
    sort=True,
    sort_key=lambda x: len(x.text),
    repeat=False)

# Model

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment7.png" width="450">

In [22]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        
        output, hidden = self.rnn(embedded)

        return self.fc(hidden.squeeze(0))

In [23]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

INPUT_DIM = len(TEXT.vocab.stoi)
EMBEDDING_DIM = 256
HIDDEN_DIM = 64
OUTPUT_DIM = 2

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)

def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [24]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.text)

        loss = criterion(predictions, batch.label)
        acc = get_accuracy(predictions, batch.label, BATCH_SIZE)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)

            loss = criterion(predictions, batch.label)
            acc = get_accuracy(predictions, batch.label, BATCH_SIZE)

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [26]:
N_EPOCHS = 15

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc:.2f}%')

Epoch: 01, Train Loss: 0.654, Train Acc: 61.01%, Val. Loss: 0.704, Val. Acc: 50.75%
Epoch: 02, Train Loss: 0.641, Train Acc: 62.49%, Val. Loss: 0.706, Val. Acc: 50.62%
Epoch: 03, Train Loss: 0.637, Train Acc: 62.80%, Val. Loss: 0.705, Val. Acc: 50.61%
Epoch: 04, Train Loss: 0.633, Train Acc: 62.99%, Val. Loss: 0.702, Val. Acc: 50.80%
Epoch: 05, Train Loss: 0.620, Train Acc: 65.08%, Val. Loss: 0.680, Val. Acc: 57.22%
Epoch: 06, Train Loss: 0.577, Train Acc: 69.58%, Val. Loss: 0.682, Val. Acc: 61.83%
Epoch: 07, Train Loss: 0.545, Train Acc: 72.23%, Val. Loss: 0.737, Val. Acc: 60.57%
Epoch: 08, Train Loss: 0.554, Train Acc: 72.08%, Val. Loss: 0.614, Val. Acc: 68.74%
Epoch: 09, Train Loss: 0.494, Train Acc: 76.70%, Val. Loss: 0.659, Val. Acc: 62.53%
Epoch: 10, Train Loss: 0.468, Train Acc: 78.16%, Val. Loss: 0.583, Val. Acc: 71.72%
Epoch: 11, Train Loss: 0.430, Train Acc: 81.07%, Val. Loss: 0.561, Val. Acc: 74.12%
Epoch: 12, Train Loss: 0.428, Train Acc: 81.72%, Val. Loss: 0.658, Val. Acc:

In [27]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

Test Loss: 0.537, Test Acc: 76.61%
